# 1. Data Preparation
## 1.1 Japan House Price dataset

In [ ]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score

# Set random seed
seed = 22
np.random.seed(seed)
random.seed(seed)

# Set display options to prevent auto line wrapping
pd.set_option('display.expand_frame_repr', False)  

file_path = 'All_prefectures_buildings_with_migration.csv'  
data = pd.read_csv(file_path)

print("data info:")
print(data.info())
print("\ndata head:")
print(data.head())
print("\ndata distribution:")
print(data.describe())
print("\n Year count:")
print(((data['Year'] > 2013) & (data['Year'] < 2024)).sum())

data info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1339147 entries, 0 to 1339146
Data columns (total 32 columns):
 #   Column                          Non-Null Count    Dtype  
---  ------                          --------------    -----  
 0   Type                            1339147 non-null  object 
 1   City,Town,Ward,Village code     1339147 non-null  int64  
 2   Prefecture                      1339147 non-null  object 
 3   Location                        1339147 non-null  object 
 4   TotalTransactionValue           1339147 non-null  int64  
 5   Area                            1339147 non-null  int64  
 6   Frontage                        1339147 non-null  float64
 7   TotalFloorArea                  1339147 non-null  int64  
 8   ConstructionYear                1339147 non-null  int64  
 9   BuildingCoverageRatio           1339147 non-null  float64
 10  FloorAreaRatio                  1339147 non-null  float64
 11  Quarter                         1339147 non-null  in

In [ ]:
# filter data from 2013 to 2024
data = data[(data['Year'] > 2013) & (data['Year'] < 2024)]
data = data[(data["Region_Kanto"]== True)]

print("data info:")
print(data.info())
print("\ndata head:")
print(data.head())
print("\ndata distribution:")
print(data.describe())
print(data['Year'].value_counts())

data info:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 317229 entries, 202846 to 956044
Data columns (total 32 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   Type                            317229 non-null  object 
 1   City,Town,Ward,Village code     317229 non-null  int64  
 2   Prefecture                      317229 non-null  object 
 3   Location                        317229 non-null  object 
 4   TotalTransactionValue           317229 non-null  int64  
 5   Area                            317229 non-null  int64  
 6   Frontage                        317229 non-null  float64
 7   TotalFloorArea                  317229 non-null  int64  
 8   ConstructionYear                317229 non-null  int64  
 9   BuildingCoverageRatio           317229 non-null  float64
 10  FloorAreaRatio                  317229 non-null  float64
 11  Quarter                         317229 non-null  int64  
 12  

In [3]:
# drop some columns
columns_to_drop = [
    "Type",
    "City,Town,Ward,Village code",
    "Location",
    "Quarter",
    "FloorAreaGreaterFLag",
    "frontage_greater_than_50",
    "AreaGreaterFlag",
    "Region_Chubu",
    "Region_Chugoku",
    "Region_Hokkaido",
    "Region_Kansai",
    "Region_Kyushu",
    "Region_Shikoku",
    "Region_Tohoku",
    "Region_Kanto"
]

data = data.drop(columns=columns_to_drop)

# save the new dataset
data.to_csv("kanto_house_price_cleaned_2014_2023.csv", index=False)

In [4]:
# Check unique value
print(data.nunique())

print(data['Prefecture'].value_counts().sort_index())

Prefecture                           7
TotalTransactionValue              428
Area                               127
Frontage                           471
TotalFloorArea                     129
ConstructionYear                    81
BuildingCoverageRatio                6
FloorAreaRatio                      13
Year                                10
RegionCommercialArea                 2
RegionIndustrialArea                 2
RegionPotentialResidentialArea       2
RegionResidentialArea                2
MunicipalityCategory                 4
AverageTimeToStation                34
BeforeWarFlag                        2
Migration                         2985
dtype: int64
Chiba Prefecture       51581
Gunma Prefecture       12163
Ibaraki Prefecture     18945
Kanagawa Prefecture    73186
Saitama Prefecture     61639
Tochigi Prefecture     14606
Tokyo                  85109
Name: Prefecture, dtype: int64


In [ ]:
# check the unique value for catregorical variables
for col in data.select_dtypes(include='object').columns:
    print(f"\n'{col}' unique value distribution:")
    print(data[col].value_counts())
    
# check the distribution of bool variables
for col in data.select_dtypes(include='bool').columns:
    print(f"\n'{col}' Distribution:")
    print(data[col].value_counts())


'Prefecture' unique value distribution:
Tokyo                  85109
Kanagawa Prefecture    73186
Saitama Prefecture     61639
Chiba Prefecture       51581
Ibaraki Prefecture     18945
Tochigi Prefecture     14606
Gunma Prefecture       12163
Name: Prefecture, dtype: int64

'RegionCommercialArea' Distribution:
False    308650
True       8579
Name: RegionCommercialArea, dtype: int64

'RegionIndustrialArea' Distribution:
False    316931
True        298
Name: RegionIndustrialArea, dtype: int64

'RegionPotentialResidentialArea' Distribution:
False    317207
True         22
Name: RegionPotentialResidentialArea, dtype: int64

'RegionResidentialArea' Distribution:
True     308330
False      8899
Name: RegionResidentialArea, dtype: int64

'BeforeWarFlag' Distribution:
False    316990
True        239
Name: BeforeWarFlag, dtype: int64


## 1.2 Add World Bank dataset

In [ ]:
import os

# add World Bank data
folder_path = "./WorldBankJP"
output_list = []

indicators_to_keep = {
    "Inflation, consumer prices (annual %)": "InflationRate",
    "Unemployment, total (% of total labor force) (modeled ILO estimate)": "UnemploymentRate",
    "GDP growth (annual %)": "GDPGrowth",
    "GDP per capita (constant LCU)": "GDPPerCapita",
    "Population, total": "Population",
    "Net migration": "NetMigration",
    "Age dependency ratio, old (% of working-age population)": "OldAgeDependencyRatio"
}

for file_name in os.listdir(folder_path):
    if file_name.endswith(".csv"):
        df = pd.read_csv(os.path.join(folder_path, file_name))
        japan_row = df[df["Country Name"] == "Japan"]

        if not japan_row.empty:
            year_cols = [str(y) for y in range(2014, 2024)]
            target_cols = ["Indicator Name"] + [col for col in year_cols if col in japan_row.columns]
            japan_filtered = japan_row[target_cols]
            japan_filtered = japan_filtered[japan_filtered["Indicator Name"].isin(indicators_to_keep.keys())]
            japan_filtered["Indicator Name"] = japan_filtered["Indicator Name"].map(indicators_to_keep)
            output_list.append(japan_filtered)

if output_list:
    final_df = pd.concat(output_list, ignore_index=True)
    final_df.to_csv("japan_indicators_filtered.csv", index=False)
    print("Extraction completed.")
else:
    print("Error.")

Extraction completed.


# 2. KHPI Dataset

In [ ]:
# read csv files
ind_df = pd.read_csv("japan_indicators_filtered.csv")
house_df = pd.read_csv("kanto_house_price_cleaned_2014_2023.csv")

# Transpose the DataFrame and reset column names
ind_df.set_index("Indicator Name", inplace=True)
ind_df = ind_df.transpose()
ind_df.index.name = "Year"
ind_df.reset_index(inplace=True)

# Convert the 'Year' column to integer for merging
ind_df["Year"] = ind_df["Year"].astype(int)

# Merge the two DataFrames on 'Year'
merged_df = pd.merge(house_df, ind_df, on="Year", how="left")

# Save the result
merged_df.to_csv("kanto_house_price_with_indicators.csv", index=False)
print("Merge completed.")

Merge completed.


In [8]:
file_path = 'kanto_house_price_with_indicators.csv'  
data = pd.read_csv(file_path)

print("data info:")
print(data.info())
print("\ndata head:")
print(data.head())
print("\ndata distribution:")
print(data.describe())

data info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 317229 entries, 0 to 317228
Data columns (total 24 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   Prefecture                      317229 non-null  object 
 1   TotalTransactionValue           317229 non-null  int64  
 2   Area                            317229 non-null  int64  
 3   Frontage                        317229 non-null  float64
 4   TotalFloorArea                  317229 non-null  int64  
 5   ConstructionYear                317229 non-null  int64  
 6   BuildingCoverageRatio           317229 non-null  float64
 7   FloorAreaRatio                  317229 non-null  float64
 8   Year                            317229 non-null  int64  
 9   RegionCommercialArea            317229 non-null  bool   
 10  RegionIndustrialArea            317229 non-null  bool   
 11  RegionPotentialResidentialArea  317229 non-null  bool   
 12  Regio

In [9]:
# Check unique value
print(data.nunique())

Prefecture                           7
TotalTransactionValue              428
Area                               127
Frontage                           471
TotalFloorArea                     129
ConstructionYear                    81
BuildingCoverageRatio                6
FloorAreaRatio                      13
Year                                10
RegionCommercialArea                 2
RegionIndustrialArea                 2
RegionPotentialResidentialArea       2
RegionResidentialArea                2
MunicipalityCategory                 4
AverageTimeToStation                34
BeforeWarFlag                        2
Migration                         2985
InflationRate                       10
NetMigration                         9
GDPPerCapita                        10
Population                          10
UnemploymentRate                     9
GDPGrowth                           10
OldAgeDependencyRatio               10
dtype: int64
